# Скрипт для прогрева кошельков в EVM-сетях

Идея скрипта @S7miles. Сообщества https://t.me/sybilders и https://t.me/shitbilders. Купить готовый софт http://t.me/sybilders_bot.
            
            

## Что делает этот скрипт

Для работы используются:
- txt файл с приватными ключалми от кошельков, для которых будут осуществляться транзакции,
- модуль chain_settings.py в котором задаются настройки для сетей, в которых будут осуществляться транзакции.

Используются два вида транзакций:
- SELF - отправка идет на свой адрес,
- RANDOM - генерируется случайный адрес, на него отправляется некоторое количество монет.

Для каждого кошелька из списка берется сеть, определяется рандомное количество транзакций, в рамках этого количества рандомно выбирается вид транзакции. Например, в сети BSC будет совершено 12 транзакций, из них 3 будет SELF, а 9 - RANDOM и т.п. После каждой транзакции процесс засыпает на время, которое определяется рандомно.

После завершения всех транзакций для всех адресов, экспортируется отчет в csv.

# Синхронный вариант кода

Все инструкции выполняются по очереди.

## Модуль chain_settings.py

В файле в виде словарей перечислены сети, в которых будут совершаться транзакции.

Настройки сетей:<br>
**name** - название сети, использует для вывода в консоль и в отчете<br>
**url** - используется для подключения<br>
<br>

**min_amount_for_sending** - минимальная сумма для отправки<br>
**max_amount_for_sending** - максимальная сумма для отправки<br>
Из диапазона этих чисел будет определено случайное число для отправки.
<br>
<br>
**min_transactions** - минимальное количество транзакций, которое должно быть совершено в сети<br>
**max_transactions** - максимально<br>
Из диапазона этих чисел будет определено случайное количество транзакций.
<br>
<br>
**explorer_url** - url эксплорера, к которому будет подставлен хэш транзакции<br>
<br>
<br>
Можно убрать параметры, если они избыточны, или добавить новые.

In [9]:
polygon = {
            'name': 'Polygon',
            'url':'https://polygon.llamarpc.com',
            'min_amount_for_sending':0.0001,
            'max_amount_for_sending':0.001,
            'min_transactions':5,
            'max_transactions':10,
            'explorer_url':'https://polygonscan.com/tx/'
            }

In [10]:
bsc = {
            'name': 'Binance Smart Chain',
            'url':'https://bsc.publicnode.com',
            'min_amount_for_sending':0.0001,
            'max_amount_for_sending':0.001,
            'min_transactions':5,
            'max_transactions':10,
            'explorer_url':'https://bscscan.com/tx/'
            }

In [11]:
avalanche = {
            'name': 'Avalanche',
            'url':'https://avalanche-c-chain.publicnode.com',
            'min_amount_for_sending':0.0001,
            'max_amount_for_sending':0.001,
            'min_transactions':5,
            'max_transactions':10,
            'explorer_url':'https://snowtrace.io/tx/'
            }

После того, как вы задали пресеты для сетей, в список _ _all_ _ передайте названия сетей (словари), которые будут использоваться для отправки.

In [12]:
__all__ = ['polygon', 'bsc', 'avalanche']

## Config.py

Название файла с приватниками:

In [13]:
TXT_WITH_PRIVATE_KEYS = 'private_keys.txt'

Название для таблицы с отчетом, которая будет создана:

In [14]:
REPORT_TABLE_NAME = 'report_table.csv'

Количество вариаций, которое будет в диапазоне между min_amount_for_sending и max_amount_for_sending - в настройках chain_settings.py. Из всех вариантов будет рандомно выбрана сумма для отправки:

In [15]:
NUMBER_OF_VARIATION_BETWEEN_VALUES = 50

Минимальная задержка между транзакциями

In [16]:
MIN_TIMESLEEP_BETWEEN_TRANSACTIONS = 100

Максимальная задержка между транзакциями

In [17]:
MAX_TIMESLEEP_BETWEEN_TRANSACTIONS = 250

Из этого диапазона будет выбрано рандомное значение, на которое скрипт остановится перед тем, как сделать новую транзакцию. Ставьте значения побольше, чтобы скрипт не делал несколько транзакций за минуту.

## Модуль functions.py

### Библиотеки

In [18]:
import time
import random
from datetime import datetime

import numpy as np
import pandas as pd

from web3 import Web3

from config import REPORT_TABLE_NAME, NUMBER_OF_VARIATIONS_BETWEEN_SENDING_AMOUNT, MIN_TIMESLEEP_BETWEEN_TRANSACTIONS, \
    MAX_TIMESLEEP_BETWEEN_TRANSACTIONS
import chain_settings


**time** - для задержек<br>
**random** - для рандомизации<br>
**datetime** - дата, которая испоьзуется в отчете<br>
**pandas** - инструмент для работы с таблицами, используется для загрузки/создания таблицы с отчетом, для записи данных<br>
**numpy** - используется для создания диапазона из любых видов чисел - целых или десятичных<br>
**web3** - подключение к rpc, создание, подпись и отправка транзакций<br>
**config** - из файла явно импортируются переменные<br>
**chain_settings** - модуль с настройками чейнов, в которых будут совершаться транзы<br>

### Создание отчета

Для отчетов создается пустой лист:

In [19]:
global_list_for_report = list()

#### Создание отчета

Принимает название чейна, публичный адрес кошелька, тип транзакции, статус транзакции, количество отправленных монет, url для просмотра транзакции, ошибку. Все записывается в словарь, затем добавляется в список global_list_for_report:

In [20]:
def save_report_to_global_list(chain_name, wallet_address,
                               transaction_type, transaction_status,
                               amount, transaction_explorer_url, error):
    d = dict()
    d['chain_name'] = chain_name
    d['wallet_address'] = wallet_address
    d['transaction_type'] = transaction_type
    d['transaction_status'] = transaction_status
    d['sent_amount'] = amount
    d['transaction_url'] = transaction_explorer_url
    d['date'] = datetime.now().strftime('%d-%m-%Y')
    d['error'] = error

    global global_list_for_report
    global_list_for_report.append(d)

Экспорт отчета в csv. Из списка со словарями global_list_for_report создается датафрейм и экспортируется в csv. Возможно не эргономично использовать мощный pandas для такой задачи, но код очень удобный:)

In [21]:
def export_report():
    global global_list_for_report
    df = pd.DataFrame.from_dict(global_list_for_report)
    df.to_csv(REPORT_TABLE_NAME, index=False)

## Рандомный выбор значений

Функция, которая возвращает рандомное значение из диапазона:

In [22]:
def create_random_value_from_range(min_value, max_value):
    random_value_between_range = random.choice(range(min_value, max_value))

    return random_value_between_range

### Рандомный выбор количества монет для отправки

Принимает минимальное и максимальное значение из настроек чейнов и количество вариаций. Для этой задачи используется numpy linspace, который создает диапазон из чисел с плавающей точкой, чтобы можно было отправлять маленькие суммы:

In [23]:
def create_random_amount_for_sending(min_value_for_sending, max_value_for_sending,
                                     number_of_variations_between_sending_amount):
    random_position = random.choice(range(number_of_variations_between_sending_amount))

    random_amount_for_sending = np.linspace(min_value_for_sending,
                                            max_value_for_sending,
                                            num=number_of_variations_between_sending_amount)[random_position]

    return random_amount_for_sending

numpy linspace возвращает количество значений в диапазоне

In [24]:
np.linspace(0.001, 1, num=50)

array([0.001     , 0.02138776, 0.04177551, 0.06216327, 0.08255102,
       0.10293878, 0.12332653, 0.14371429, 0.16410204, 0.1844898 ,
       0.20487755, 0.22526531, 0.24565306, 0.26604082, 0.28642857,
       0.30681633, 0.32720408, 0.34759184, 0.36797959, 0.38836735,
       0.4087551 , 0.42914286, 0.44953061, 0.46991837, 0.49030612,
       0.51069388, 0.53108163, 0.55146939, 0.57185714, 0.5922449 ,
       0.61263265, 0.63302041, 0.65340816, 0.67379592, 0.69418367,
       0.71457143, 0.73495918, 0.75534694, 0.77573469, 0.79612245,
       0.8165102 , 0.83689796, 0.85728571, 0.87767347, 0.89806122,
       0.91844898, 0.93883673, 0.95922449, 0.97961224, 1.        ])

Из диапазона, который равен количеству вариаций выбирается число:

In [25]:
random.choice(range(50))

22

Которое используется в качестве случайного индекса для выбора числа из numpy linspace

In [27]:
np.linspace(0.001, 1, num=50)[22]

0.449530612244898

Такое количество монет будет использовано в транзакции

## Транзакции

### Настройка транзакции

Принимает подключение, адрес отправителя, адрес получателя, количество монет для отправки.<br>
В amount_wei конвертируется отправляемое значение из ether в wei.<br>
В gas_price определяется цена газа, в estimated_gas оценка лимита газа для отправки крипты.<br>
В nonce определяется номер транзакции.<br>
В словаре transaction записываются все параметры будущей транзакции.<br>
Функция возращает словарь transaction.

In [28]:
def transaction_settings(web3, from_address, to_address, amount):
    amount_wei = web3.to_wei(amount, 'ether')

    gas_price = web3.eth.gas_price

    estimated_gas = web3.eth.estimate_gas({
        'to': to_address,
        'value': amount_wei,
    })

    nonce = web3.eth.get_transaction_count(from_address)

    transaction = {
        'chainId': web3.eth.chain_id,
        'from': from_address,
        'to': to_address,
        'value': amount_wei,
        'nonce': nonce,
        'gasPrice': gas_price,
        'gas': estimated_gas,
    }

    return transaction

### Подпись и отправка транзакции

Принимает подключение, адрес отправителя, адрес получателя, количество монет для отправки, приватный ключ.<br>
Настройки будущей транзы записываются в create_transaction.<br>
В signed_transaction передаются настройки и приватный ключ, транзакция подписывается.<br>
В transaction_hash записывается хэш отправленной транзакции.<br>
Функция возвращает хэш.

In [29]:
def create_sign_send_transaction(web3, from_address, to_address, amount, private_key):
    create_transaction = transaction_settings(
        web3=web3,
        from_address=from_address,
        to_address=to_address,
        amount=amount
    )

    signed_transaction = web3.eth.account.sign_transaction(create_transaction, private_key)

    transaction_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)

    return transaction_hash.hex()

### SELF транзакция

Принимает подключение, количество монет для отправки, приватник, минимальную и максимальную задержку между транзакциями, название чейна и url эксплорера.<br>
<br>
Перед отправкой в timesleep_value генерируется рандомное число из небольшого диапазона, скрипт останавливается на это время.<br>
<br>
В my_address записывается публичный адрес, который определяется из приватного ключа.<br>
<br>
В create_sign_send_transaction осуществляется отправка, в качестве отправителя и получателя передается my_address. Хэш записывается в transaction_hash.<br>
<br>
В save_report_to_global_list записывается отчет, в функцию передается название сети, публичный адрес, тип транзакции, статус SUCCESS, сумма, url, который получается в результате подстановки параметра из chain_settings.py и хэша транзы, ошибки нет, поэтому передается прочерк.<br>
<br>
В timesleep_value генерируется рандомное число из переданных в функцию значений. Скрипт засыпает.<br>
<br>
Если срабатывает блок except, в отчет записывается название сети, публичный адрес, тип транзакции, статус ERROR, сумма, вместо url передается прочерк, код ошибки.

In [30]:
def self_transaction(web3, amount, private_key,
                     min_timesleep_between_transactions,
                     max_timesleep_between_transactions,
                     chain_name, explorer_url):

    try:
        timesleep_value = create_random_value_from_range(5, 15)
        time.sleep(timesleep_value)

        transaction_type = 'SELF-transaction'
        print(f'Key {private_key[:6]}...{private_key[-6:]}, {transaction_type} {chain_name}')

        my_address = web3.eth.account.from_key(private_key).address

        transaction_hash = create_sign_send_transaction(web3, my_address, my_address,
                                                        amount, private_key)

        print(f'Hash {transaction_type} {chain_name} {transaction_hash}')

        print(f'Recording data in a report')
        save_report_to_global_list(chain_name, my_address,
                                   transaction_type, 'SUCCESS',
                                   amount, f'{explorer_url}{transaction_hash}', '-')

        timesleep_value = create_random_value_from_range(min_timesleep_between_transactions,
                                                         max_timesleep_between_transactions)
        print(f'Pause {timesleep_value} sec. after {transaction_type} {chain_name} ', '\n')
        time.sleep(timesleep_value)

    except Exception as error:
        print(f'{transaction_type} {chain_name}, error -  {error}', '\n')
        save_report_to_global_list(chain_name, my_address,
                                   transaction_type, 'ERROR',
                                   amount, '-', error)

### RANDOM транзакция

Почти такая же функция как self_transaction, отличается наличием кода на 12-16 строках.<br>
<br>
Принимает подключение, количество монет для отправки, приватник, минимальную и максимальную задержку между транзакциями, название чейна и url эксплорера.<br>
<br>
Перед отправкой в timesleep_value генерируется рандомное число из небольшого диапазона, скрипт останавливается на это время.<br>
<br>
В account создается новый аккаунт. В random_wallet записывается публичный адрес созданного аккаунта, на который будет осуществляться отправка.<br>
<br>
В my_address записывается публичный адрес, который определяется из приватного ключа.<br>
<br>
В prepared_wallet записывается созданный публичный адрес, приведенный к корректному виду с помощью to_checksum_address.<br>
<br>
В create_sign_send_transaction осуществляется отправка, в качестве отправителя и получателя передается my_address. Хэш записывается в transaction_hash.<br>
<br>
В save_report_to_global_list записывается отчет, в функцию передается название сети, публичный адрес, тип транзакции, статус SUCCESS, сумма, url, который получается в результате подстановки параметра из chain_settings.py и хэша транзы, ошибки нет, поэтому передается прочерк.<br>
<br>
В timesleep_value генерируется рандомное число из переданных в функцию значений. Скрипт засыпает.<br>
<br>
Если срабатывает блок except, в отчет записывается название сети, публичный адрес, тип транзакции, статус ERROR, сумма, вместо url передается прочерк, код ошибки.

In [31]:
def random_transaction(web3, amount, private_key,
                       min_timesleep_between_transactions,
                       max_timesleep_between_transactions,
                       chain_name, explorer_url):
    try:
        timesleep_value = create_random_value_from_range(5, 15)
        time.sleep(timesleep_value)

        transaction_type = 'RANDOM-transaction'
        print(f'Key {private_key[:6]}...{private_key[-6:]}, {transaction_type} {chain_name}', '\n')

        account = web3.eth.account.create()
        random_wallet = account.address

        my_address = web3.eth.account.from_key(private_key).address
        prepared_wallet = web3.to_checksum_address(random_wallet)

        transaction_hash = create_sign_send_transaction(web3, my_address, prepared_wallet,
                                                        amount, private_key)

        print(f'Hash {transaction_type} {chain_name} {transaction_hash}' )

        print(f'Recording data in a report')
        save_report_to_global_list(chain_name, my_address,
                                   transaction_type, 'SUCCESS',
                                   amount, f'{explorer_url}{transaction_hash}', '-')

        timesleep_value = create_random_value_from_range(min_timesleep_between_transactions,
                                                         max_timesleep_between_transactions)
        print(f'Pause {timesleep_value} sec. after {transaction_type} {chain_name} ', '\n')
        time.sleep(timesleep_value)

    except Exception as error:
        print(f'{transaction_type} {chain_name}, error -  {error}', '\n')
        save_report_to_global_list(chain_name, my_address,
                                   transaction_type, 'ERROR',
                                   amount, '-', error)

### Главная функция

Принимает приватник.<br>
<br>
Запускается цикл, который перебирает словари из chain_settigs.py, указанные в _ _ all _ _<br>
<br>

В переменную web3 записывается объект созданного подключения. Если подключение False (if not web3.is_connected()), процесс останавливается. Записывается в отчет. Для отчета в этой части кода генерируется публичный адрес my_address из приватника.<br>
<br>
В transactions_quantity записывается рандомное количество транзакций, которое будет совершено в сети. Число генерируется из диапазона, указанного в настройках в chain_settigs.py.<br>
<br>
Запускается цикл по количеству транзакций в значении transactions_quantity.<br>
<br>
В amount определяется случайная сумма к отправке, генерируется функцией create_random_amount_for_sending, которая берет диапазон из настроек chain_settigs.py и количество вариаций внутри этого диапазона NUMBER_OF_VARIATIONS_BETWEEN_SENDING_AMOUNT из файла config.py.<br>
<br>
На каждой итерации цикла выбирается SELF или RANDOM транзакция.<br>
<br>
Если срабатывает блок except, записывается отчет с пустыми параметрами (прочерками), с приватным ключом и сообщением, что возникла системная ошибка.

In [32]:
def main_function_make_random_or_self_txns(private_key):
    try:
        for chain in map(chain_settings.__dict__.get, chain_settings.__all__):
            chain_name = chain['name']

            print(f'Process has been launched for {private_key[:6]}...{private_key[-6:]} on the network {chain_name}')

            web3 = Web3(Web3.HTTPProvider(chain['url']))

            if not web3.is_connected():
                print(f'Connection to {chain_name} is not established', '\n')

                my_address = web3.eth.account.from_key(private_key).address
                save_report_to_global_list(chain_name, my_address,
                                           '-', 'ERROR',
                                           '-', '-',
                                           'Connection is not established')
                time.sleep(5)
                continue

            print(f'Connection to {chain_name} was established', '\n')

            transactions_quantity = create_random_value_from_range(chain['min_transactions'],
                                                                   chain['max_transactions'])

            for _ in range(transactions_quantity):
                amount = create_random_amount_for_sending(chain['min_amount_for_sending'],
                                                          chain['max_amount_for_sending'],
                                                          NUMBER_OF_VARIATIONS_BETWEEN_SENDING_AMOUNT)

                random.choice([self_transaction, random_transaction])(web3, amount, private_key,
                                                                      MIN_TIMESLEEP_BETWEEN_TRANSACTIONS,
                                                                      MAX_TIMESLEEP_BETWEEN_TRANSACTIONS,
                                                                      chain_name, chain['explorer_url'])

    except Exception as error:
        print(error, '\n')
        save_report_to_global_list('-', private_key,
                                   '-', 'ERROR',
                                   '-', '-',
                                   'Some system error occurred during the process.')

## Модуль main.py

Из модуля с functions импортируется главная функция и функция для создания отчета. Из файла config импортируется переменная с названием txt файла с приватными ключами:

In [34]:
from functions import main_function_make_random_or_self_txns, export_report
from config import TXT_WITH_PRIVATE_KEYS

В главной функции в список list_of_private_keys добавляются все приватные ключи из txt-файла. В цикле каждый ключ передается в функцию main_function_make_random_or_self_txns, в которой выпоняется весь процесс:

In [35]:
def main():
    with open(TXT_WITH_PRIVATE_KEYS) as file:
        list_of_private_keys = [line.strip() for line in file.readlines()]

    for private_key in list_of_private_keys:
        main_function_make_random_or_self_txns(private_key)

    export_report()

Точка входа:

In [36]:
if __name__ == '__main__':
    main()

Process has been launched for 4b63b8...cc8bc3 on the network Binance Smart Chain
Connection to Binance Smart Chain was established 

Key 4b63b8...cc8bc3, RANDOM-transaction Binance Smart Chain 

Hash RANDOM-transaction Binance Smart Chain 0x08dc9dc4f370234f1ab7da0055467aff7bb6a57a23819d86e32f339487fd1ea3
Recording data in a report
Pause 8 sec. after RANDOM-transaction Binance Smart Chain  

Key 4b63b8...cc8bc3, SELF-transaction Binance Smart Chain
Hash SELF-transaction Binance Smart Chain 0x2e0d1b81fdb13a99641152aa663da43145fde0c5851144775158c22b3b620de0
Recording data in a report
Pause 6 sec. after SELF-transaction Binance Smart Chain  

Key 4b63b8...cc8bc3, RANDOM-transaction Binance Smart Chain 

Hash RANDOM-transaction Binance Smart Chain 0x3ebb6e5de811a39f53e25251dfc347c1131060648f41ababc51a58965b4d607b
Recording data in a report
Pause 8 sec. after RANDOM-transaction Binance Smart Chain  

Process has been launched for 4b63b8...cc8bc3 on the network Polygon
Connection to Polygon w

KeyboardInterrupt: 

# Асинхронный вариант кода

В этом варианте для каждого вашего кошелька процессы будут выполняться одновременно. Чтобы управлять процессами реализована очередь, которая ограничивается параметром в настройках. Очередь заполняется продюсерами (producer), которые одновременно с другими процессами наполняют очередь до указанных в настройках размеров. Процессы с транзакциями выполняются консюмерами (consumer), или их можно назвать воркерами. Их количество тоже ограничивается в настройках.

## Модуль chain_settings.py

**Идентичный файл как в синхронном варианте, описанном выше.**

В файле в виде словарей перечислены сети, в которых будут совершаться транзакции.

Настройки сетей:<br>
**name** - название сети, использует для вывода в консоль и в отчете<br>
**url** - используется для подключения<br>
<br>

**min_amount_for_sending** - минимальная сумма для отправки<br>
**max_amount_for_sending** - максимальная сумма для отправки<br>
Из диапазона этих чисел будет определено случайное число для отправки.
<br>
<br>
**min_transactions** - минимальное количество транзакций, которое должно быть совершено в сети<br>
**max_transactions** - максимально<br>
Из диапазона этих чисел будет определено случайное количество транзакций.
<br>
<br>
**explorer_url** - url эксплорера, к которому будет подставлен хэш транзакции<br>
<br>
<br>
Можно убрать параметры, если они избыточны, или добавить новые.

In [51]:
polygon = {
            'name': 'Polygon',
            'url':'https://polygon.llamarpc.com',
            'min_amount_for_sending':0.0001,
            'max_amount_for_sending':0.001,
            'min_transactions':5,
            'max_transactions':10,
            'explorer_url':'https://polygonscan.com/tx/'
            }

In [52]:
bsc = {
            'name': 'Binance Smart Chain',
            'url':'https://bsc.publicnode.com',
            'min_amount_for_sending':0.0001,
            'max_amount_for_sending':0.001,
            'min_transactions':5,
            'max_transactions':10,
            'explorer_url':'https://bscscan.com/tx/'
            }

In [53]:
avalanche = {
            'name': 'Avalanche',
            'url':'https://avalanche-c-chain.publicnode.com',
            'min_amount_for_sending':0.0001,
            'max_amount_for_sending':0.001,
            'min_transactions':5,
            'max_transactions':10,
            'explorer_url':'https://snowtrace.io/tx/'
            }

После того, как вы задали пресеты для сетей, в список _ _all_ _ передайте названия сетей (словари), которые будут использоваться для отправки.

In [54]:
__all__ = ['polygon', 'bsc', 'avalanche']

## Config.py

**В отличии от синхронного варианта содержит дополнительные насройки для управления асинхронными процессами.**

Название файла с приватниками:

In [55]:
TXT_WITH_PRIVATE_KEYS = 'private_keys.txt'

Название для таблицы с отчетом, которая будет создана:

In [56]:
REPORT_TABLE_NAME = 'report_table.csv'

Количество вариаций, которое будет в диапазоне между min_amount_for_sending и max_amount_for_sending - в настройках chain_settings.py. Из всех вариантов будет рандомно выбрана сумма для отправки:

In [57]:
NUMBER_OF_VARIATION_BETWEEN_VALUES = 50

Минимальная задержка между транзакциями:

In [58]:
MIN_TIMESLEEP_BETWEEN_TRANSACTIONS = 100

Максимальная задержка между транзакциями:

In [59]:
MAX_TIMESLEEP_BETWEEN_TRANSACTIONS = 250

Из этого диапазона будет выбрано рандомное значение, на которое скрипт остановится перед тем, как сделать новую транзакцию. Ставьте значения побольше, чтобы скрипт не делал несколько транзакций за минуту.

Количество одновременно работающих процессов (консюмеров/воркеров):

In [60]:
NUMBER_OF_PROCESSES = 4

Перед стартом каждого процесса есть небольшая задержка. Число рандомно генерируется из диапазона:

In [61]:
MIN_TIMESLEEP_BETWEEN_PROCESSES = 50

In [62]:
MAX_TIMESLEEP_BETWEEN_PROCESSES = 100

Список кодов цветов, которыми будут подсвечены разные процессы в терминале, чтобы чуть легче было их наблюдать:

In [63]:
colors = ['\033[32m',
          '\033[33m',
          '\033[34m',
          '\033[35m',
          '\033[36m',
          '\033[37m']

## Модуль functions.py

### Библиотеки

In [38]:
import asyncio
import random
from datetime import datetime

import numpy as np
import pandas as pd

from config import REPORT_TABLE_NAME

**asyncio** - библиотека для асинхронного программирования<br>
**random** - для рандомизации<br>
**datetime** - дата, которая испоьзуется в отчете<br>
**pandas** - инструмент для работы с таблицами, используется для загрузки/создания таблицы с отчетом, для записи данных<br>
**numpy** - используется для создания диапазона из любых видов чисел - целых или десятичных<br>
**config** - из файла явно импортируется название csv для отчета<br>

### Создание отчета

Для отчетов создается пустой лист:

In [19]:
global_list_for_report = list()

Создается замок:

In [39]:
lock = asyncio.Lock()

#### Создание отчета

Принимает название чейна, публичный адрес кошелька, тип транзакции, статус транзакции, количество отправленных монет, url для просмотра транзакции, ошибку. Все записывается в словарь, затем добавляется в список global_list_for_report. Запись блокируется замком (async with lock), чтобы предотвратить одновременное записывание данных асинхронными функциями (корутинами) в список для отчета global_list_for_report:

In [40]:
async def save_report_to_global_list(chain_name, wallet_address,
                                     transaction_type, transaction_status,
                                     amount, transaction_explorer_url, error):
    async with lock:
        d = dict()
        d['chain_name'] = chain_name
        d['wallet_address'] = wallet_address
        d['transaction_type'] = transaction_type
        d['transaction_status'] = transaction_status
        d['sent_amount'] = amount
        d['transaction_url'] = transaction_explorer_url
        d['date'] = datetime.now().strftime('%d-%m-%Y')
        d['error'] = error

        global global_list_for_report
        global_list_for_report.append(d)

Экспорт отчета в csv. Из списка со словарями global_list_for_report создается датафрейм и экспортируется в csv. Возможно не эргономично использовать мощный pandas для такой задачи, но код очень удобный:)<br>
Функция остается обычной (синхронной):

In [41]:
def export_report():
    global global_list_for_report
    df = pd.DataFrame.from_dict(global_list_for_report)
    df.to_csv(REPORT_TABLE_NAME, index=False)

### Рандомный выбор значений

**Эти функции остались такими же как в варианте с синхронным кодом, описанным выше. Эти функции остались синхронными, т.к. осуществляют вычисления и не занимаются вводом выводом данных. Так же я посчитал, что их не нужно помещать в отдельный поток, т.к. они очень быстро исполняются.**

Функция, которая возвращает рандомное значение из диапазона:

In [42]:
def create_random_value_from_range(min_value, max_value):
    random_value_between_range = random.choice(range(min_value, max_value))

    return random_value_between_range

### Рандомный выбор количества монет для отправки

Принимает минимальное и максимальное значение из настроек чейнов и количество вариаций. Для этой задачи используется numpy linspace, который создает диапазон из чисел с плавающей точкой, чтобы можно было отправлять маленькие суммы:

In [43]:
def create_random_amount_for_sending(min_value_for_sending, max_value_for_sending,
                                     number_of_variations_between_sending_amount):
    random_position = random.choice(range(number_of_variations_between_sending_amount))

    random_amount_for_sending = np.linspace(min_value_for_sending,
                                            max_value_for_sending,
                                            num=number_of_variations_between_sending_amount)[random_position]

    return random_amount_for_sending

numpy linspace возвращает количество значений в диапазоне

In [44]:
np.linspace(0.001, 1, num=50)

array([0.001     , 0.02138776, 0.04177551, 0.06216327, 0.08255102,
       0.10293878, 0.12332653, 0.14371429, 0.16410204, 0.1844898 ,
       0.20487755, 0.22526531, 0.24565306, 0.26604082, 0.28642857,
       0.30681633, 0.32720408, 0.34759184, 0.36797959, 0.38836735,
       0.4087551 , 0.42914286, 0.44953061, 0.46991837, 0.49030612,
       0.51069388, 0.53108163, 0.55146939, 0.57185714, 0.5922449 ,
       0.61263265, 0.63302041, 0.65340816, 0.67379592, 0.69418367,
       0.71457143, 0.73495918, 0.75534694, 0.77573469, 0.79612245,
       0.8165102 , 0.83689796, 0.85728571, 0.87767347, 0.89806122,
       0.91844898, 0.93883673, 0.95922449, 0.97961224, 1.        ])

Из диапазона, который равен количеству вариаций выбирается число

In [25]:
random.choice(range(50))

22

Которое используется в качестве случайного индекса для выбора числа из numpy linspace

In [27]:
np.linspace(0.001, 1, num=50)[22]

0.449530612244898

Такое количество монет будет использовано в транзакции

## Транзакции

### Настройка транзакции

Принимает подключение, адрес отправителя, адрес получателя, количество монет для отправки.<br>
В amount_wei конвертируется отправляемое значение из ether в wei. await не нужен.<br>
В gas_price определяется цена газа, в estimated_gas оценка лимита газа для отправки крипты.<br>
В nonce определяется номер транзакции.<br>
В словаре transaction записываются все параметры будущей транзакции.<br>
Функция возращает словарь transaction.

In [45]:
async def transaction_settings(web3, from_address, to_address, amount):
    amount_wei = web3.to_wei(amount, 'ether')

    gas_price = await web3.eth.gas_price

    estimated_gas = await web3.eth.estimate_gas({
        'to': to_address,
        'value': amount_wei,
    })

    nonce = await web3.eth.get_transaction_count(from_address)

    transaction = {
        'chainId': await web3.eth.chain_id,
        'from': from_address,
        'to': to_address,
        'value': amount_wei,
        'nonce': nonce,
        'gasPrice': gas_price,
        'gas': estimated_gas,
    }

    return transaction

### Подпись и отправка транзакции

Принимает подключение, адрес отправителя, адрес получателя, количество монет для отправки, приватный ключ.<br>
Настройки будущей транзы записываются в create_transaction.<br>
В signed_transaction передаются настройки и приватный ключ, транзакция подписывается. await не нужен.<br>
В transaction_hash записывается хэш отправленной транзакции.<br>
Функция возвращает хэш.

In [46]:
async def create_sign_send_transaction(web3, from_address, to_address, amount, private_key):
    create_transaction = await transaction_settings(
        web3=web3,
        from_address=from_address,
        to_address=to_address,
        amount=amount
    )

    signed_transaction = web3.eth.account.sign_transaction(create_transaction, private_key)

    transaction_hash = await web3.eth.send_raw_transaction(signed_transaction.rawTransaction)

    return transaction_hash.hex()

### SELF транзакция

Принимает подключение, количество монет для отправки, приватник, минимальную и максимальную задержку между транзакциями, название чейна и url эксплорера. В отличии от синхронного варианта принимает номер процесса и кодировку цвета, чтобы обозначить исполняемый процесс в терминале.<br>
<br>
Перед отправкой в timesleep_value генерируется рандомное число из небольшого диапазона, скрипт останавливается на это время.<br>
<br>
В my_address записывается публичный адрес, который определяется из приватного ключа. await не нужен.<br>
<br>
В create_sign_send_transaction осуществляется отправка, в качестве отправителя и получателя передается my_address. Хэш записывается в transaction_hash.<br>
<br>
В save_report_to_global_list записывается отчет, в функцию передается название сети, публичный адрес, тип транзакции, статус SUCCESS, сумма, url, который получается в результате подстановки параметра из chain_settings.py и хэша транзы, ошибки нет, поэтому передается прочерк.<br>
<br>
В timesleep_value генерируется рандомное число из переданных в функцию значений. Скрипт засыпает.<br>
<br>
Если срабатывает блок except, в отчет записывается название сети, публичный адрес, тип транзакции, статус ERROR, сумма, вместо url передается прочерк, код ошибки.

In [47]:
async def self_transaction(web3, amount, private_key,
                           min_timesleep_between_transactions,
                           max_timesleep_between_transactions,
                           chain_name, explorer_url,
                           number_of_process, color):

    try:
        timesleep_value = create_random_value_from_range(5, 15)
        await asyncio.sleep(timesleep_value)

        transaction_type = 'SELF-transaction'
        print(f'{color}Process №{number_of_process}, key {private_key[:6]}...{private_key[-6:]}, {transaction_type} {chain_name}')

        my_address = web3.eth.account.from_key(private_key).address

        transaction_hash = await create_sign_send_transaction(web3, my_address, my_address,
                                                              amount, private_key)

        print(f'{color}Process №{number_of_process}, hash {transaction_type} {chain_name} {transaction_hash}')

        print(f'{color}Process №{number_of_process}, recording data in a report')
        await save_report_to_global_list(chain_name, my_address,
                                         transaction_type, 'SUCCESS',
                                         amount, f'{explorer_url}{transaction_hash}', '-')

        timesleep_value = create_random_value_from_range(min_timesleep_between_transactions,
                                                         max_timesleep_between_transactions)
        print(f'{color}Process №{number_of_process}, pause {timesleep_value} sec. after {transaction_type} {chain_name} ', '\n')
        await asyncio.sleep(timesleep_value)

    except Exception as error:
        print(f'{color}Process №{number_of_process}, {transaction_type} {chain_name}, error -  {error}', '\n')
        await save_report_to_global_list(chain_name, my_address,
                                         transaction_type, 'ERROR',
                                         amount, '-', error)

### RANDOM транзакция

Почти такая же функция как self_transaction, отличается наличием кода на 13-17 строках.<br>
<br>
Принимает подключение, количество монет для отправки, приватник, минимальную и максимальную задержку между транзакциями, название чейна и url эксплорера. В отличии от синхронного варианта принимает номер процесса и кодировку цвета, чтобы обозначить исполняемый процесс в терминале.<br>
<br>
Перед отправкой в timesleep_value генерируется рандомное число из небольшого диапазона, скрипт останавливается на это время.<br>
<br>
В account создается новый аккаунт. В random_wallet записывается публичный адрес созданного аккаунта, на который будет осуществляться отправка. await не нужен.<br>
<br>
В my_address записывается публичный адрес, который определяется из приватного ключа. await не нужен.<br>
<br>
В prepared_wallet записывается созданный публичный адрес, приведенный к корректному виду с помощью to_checksum_address. await не нужен.<br>
<br>
В create_sign_send_transaction осуществляется отправка, в качестве отправителя и получателя передается my_address. Хэш записывается в transaction_hash.<br>
<br>
В save_report_to_global_list записывается отчет, в функцию передается название сети, публичный адрес, тип транзакции, статус SUCCESS, сумма, url, который получается в результате подстановки параметра из chain_settings.py и хэша транзы, ошибки нет, поэтому передается прочерк.<br>
<br>
В timesleep_value генерируется рандомное число из переданных в функцию значений. Скрипт засыпает.<br>
<br>
Если срабатывает блок except, в отчет записывается название сети, публичный адрес, тип транзакции, статус ERROR, сумма, вместо url передается прочерк, код ошибки.

In [48]:
async def random_transaction(web3, amount, private_key,
                             min_timesleep_between_transactions,
                             max_timesleep_between_transactions,
                             chain_name, explorer_url,
                             number_of_process, color):
    try:
        timesleep_value = create_random_value_from_range(5, 15)
        await asyncio.sleep(timesleep_value)

        transaction_type = 'RANDOM-transaction'
        print(f'{color}Process №{number_of_process}, key {private_key[:6]}...{private_key[-6:]}, {transaction_type} {chain_name}', '\n')

        account = web3.eth.account.create()
        random_wallet = account.address

        my_address = web3.eth.account.from_key(private_key).address
        prepared_wallet = web3.to_checksum_address(random_wallet)

        transaction_hash = await create_sign_send_transaction(web3, my_address, prepared_wallet,
                                                              amount, private_key)

        print(f'{color}Process №{number_of_process}, hash {transaction_type} {chain_name} {transaction_hash}' )

        print(f'{color}Process №{number_of_process}, recording data in a report')
        await save_report_to_global_list(chain_name, my_address,
                                         transaction_type, 'SUCCESS',
                                         amount, f'{explorer_url}{transaction_hash}', '-')

        timesleep_value = create_random_value_from_range(min_timesleep_between_transactions,
                                                         max_timesleep_between_transactions)
        print(f'{color}Process №{number_of_process}, pause {timesleep_value} sec. after {transaction_type} {chain_name} ', '\n')
        await asyncio.sleep(timesleep_value)

    except Exception as error:
        print(f'{color}Process №{number_of_process}, {transaction_type} {chain_name}, error -  {error}', '\n')
        await save_report_to_global_list(chain_name, my_address,
                                         transaction_type, 'ERROR',
                                         amount, '-', error)

## Модуль main.py

### Библиотеки

In [50]:
import asyncio

from web3 import AsyncWeb3

from functions import *
from config import *
import chain_settings

**asyncio** - библиотека для асинхронного программирования<br>
**web3** - подключение к rpc, осуществление транзакций на блокчейнах.<br>
**config** - импорт всех настроек из модуля<br>
**functions** - импорт всех функций из модуля<br>
**chain_settings** - импорт настроек для блокчейнов из модуля

### Функция продюсера

Принимает очередь, приватный ключ, номер. <br>
Добавляет кортеж с приватным ключем и номером в очередь. На печать выводит сообщение, что приватный ключ добавлен в очередь под каким-то номером. Надпись выводится красным уветом на белом фоне:

In [64]:
async def producer(queue, private_key, number):
    await queue.put((private_key, number))
    print('\n', f'\033[31;47mPrivate key {private_key[:6]}...{private_key[-6:]} has been added to the queue. Queue №{number}\033[0m',
          '\n')

### Функция консюмера

**Почти полностью повторяет код функции main_function_make_random_or_self_txns в синхронном варианте, описанном выше.**

Принимает очередь и цвет.<br>
<br>
Запускается бесконечный цикл.<br>
<br>
queue.get() получает первый объект из очереди. Из кортежа в private_key записывается приватный ключ, в process_number записывается номер в очереди.<br>
<br>
В timesleep_value записывается рандомное значение, выранное из диапазона, который указывается в config.py. Процесс останавливается перед стартом. Таким образом все процессы стартуют в разное время.<br>
<br>
Запускается цикл, который перебирает словари из chain_settigs.py, указанные в _ _ all _ _<br>
<br>

В переменную web3 записывается объект созданного асинхронного подключения. Если подключение False (if not web3.is_connected()), процесс останавливается. Записывается в отчет. Для отчета в этой части кода генерируется публичный адрес my_address из приватника.<br>
<br>
В transactions_quantity записывается рандомное количество транзакций, которое будет совершено в сети. Число генерируется из диапазона, указанного в настройках в chain_settigs.py.<br>
<br>
Запускается цикл по количеству транзакций в значении transactions_quantity.<br>
<br>
В amount определяется случайная сумма к отправке, генерируется функцией create_random_amount_for_sending, которая берет диапазон из настроек chain_settigs.py и количество вариаций внутри этого диапазона NUMBER_OF_VARIATIONS_BETWEEN_SENDING_AMOUNT из файла config.py.<br>
<br>
На каждой итерации цикла выбирается SELF или RANDOM транзакция.<br>
<br>
queue.task_done() уведомляет очередь о том, что задача была выполнена, обращается к приватной переменной очереди Queue._unfinished_tasks, уменьшая ее значение.
<br>
<br>
Если срабатывает блок except, записывается отчет с пустыми параметрами (прочерками), с приватным ключом и сообщением, что возникла системная ошибка.
queue.task_done() уведомляет очередь о том, что задача была выполнена.


In [65]:
async def consumer(queue, color):
    while True:
        item = await queue.get()
        private_key, process_number = item

        timesleep_value = create_random_value_from_range(MIN_TIMESLEEP_BETWEEN_PROCESSES,
                                                         MAX_TIMESLEEP_BETWEEN_PROCESSES)

        print(f'{color} ▶️ Before the start of the process №{process_number} {timesleep_value} sec.', '\n')

        await asyncio.sleep(timesleep_value)

        try:
            for chain in map(chain_settings.__dict__.get, chain_settings.__all__):
                chain_name = chain['name']

                print(f'{color}Process №{process_number} has been launched for {private_key[:6]}...{private_key[-6:]} on the network {chain_name}')

                web3 = AsyncWeb3(AsyncWeb3.AsyncHTTPProvider(chain['url']))

                if not await web3.is_connected():
                    print(f'{color}Process №{process_number} connection to {chain_name} is not established', '\n')

                    my_address = web3.eth.account.from_key(private_key).address
                    await save_report_to_global_list(chain_name, my_address,
                                                     '-', 'ERROR',
                                                     '-', '-',
                                                     'Connection is not established')
                    await asyncio.sleep(5)
                    continue

                print(f'{color}Process №{process_number} connection to {chain_name} was established', '\n')

                transactions_quantity = create_random_value_from_range(chain['min_transactions'],
                                                                       chain['max_transactions'])

                for _ in range(transactions_quantity):
                    amount = create_random_amount_for_sending(chain['min_amount_for_sending'],
                                                              chain['max_amount_for_sending'],
                                                              NUMBER_OF_VARIATIONS_BETWEEN_SENDING_AMOUNT)

                    await random.choice([self_transaction, random_transaction])(web3, amount, private_key,
                                                                                MIN_TIMESLEEP_BETWEEN_TRANSACTIONS,
                                                                                MAX_TIMESLEEP_BETWEEN_TRANSACTIONS,
                                                                                chain_name, chain['explorer_url'],
                                                                                process_number, color)

            queue.task_done()

        except Exception as error:
            print(error, '\n')
            await save_report_to_global_list('-', private_key,
                                             '-', 'ERROR',
                                             '-', '-',
                                             'Some system error occurred during the process.')
            queue.task_done()

## Главная функция - событийный цикл

В queue создается очередь. В параметре maxsize задается максимальный размер очереди. В очередь помещается указанное количество элементов, когда элементы взяты в работу, в очередь снова добавляется указанное количество элементов. В моем примере размер очереди равен количеству одновременно запущенных процессов. Поэкспериментируйте с этим параметром, можете вообще его убрать.<br>
<br>
pended равен нулю, будет увеличиваться на 1 по мере добавления приватных ключей в очередь<br>
<br>
В список list_of_private_keys добавляются приватные ключи из txt файла.<br>
<br>
Создается пустой список продюсеров producers. Запускается цикл по списку с приватными ключами. pended увеличивается на 1. В task записывается объект класса Task, т.е в событийный цикл помещается задача с результатом работы функции producer, в которую передаю очередь, приватный ключ и pended - номер задачи в очереди.<br>
<br>
Создается пустой список продюсеров consumers. Количество консюмеров/воркеров ограничено параметром NUMBER_OF_PROCESSES в config.py. В task записывается объект класса Task, т.е в событийный цикл помещается задача с результатом работы функции consumer, в которую передаю очередь, код цвета, который определяется по индексу, который в свою очередь определяется случайно.<br>
<br>
В функции gather распаковывается списко задач producers. Продюсеры начинают создавать очередь. При этом в функции consumer запущен бесконечный цикл, который ожидает очередь и находится в событийном цикле благодаря create_task.
<br>
<br>
queue.join() обращается к приватной переменной Queue._unfinished_tasks и ждет, когда очередь опустеет. Выше я писал, что в функции consumer queue.task_done() уменьшает значение этой переменной.<br>
<br>
Дополнительно, чтобы скрипт не повис, можно отменить все задачи консюмеров в событийном цикле. Цикл проходит списко задач consumers и отменяет их cancel()<br>
<br>
В конце функция export_report создает из списка global_list_for_report датафрейм и записывает его в csv.

In [66]:
async def main():
    queue = asyncio.Queue(maxsize=NUMBER_OF_PROCESSES)
    pended = 0

    with open(TXT_WITH_PRIVATE_KEYS) as file:
        list_of_private_keys = [line.strip() for line in file.readlines()]

    producers = []
    for private_key in list_of_private_keys:
        pended += 1
        task = asyncio.create_task(producer(queue, private_key, pended))
        producers.append(task)

    consumers = []
    for _ in range(NUMBER_OF_PROCESSES):
        task = asyncio.create_task(consumer(queue, colors[random.choice(range(len(colors)))]))
        consumers.append(task)

    await asyncio.gather(*producers)
    await queue.join()

    for c in consumers:
        c.cancel()

    export_report()

Точка входа:

**Чтобы запустить асинхронный скрипт в ipynb используйте этот костыль:**

In [68]:
import nest_asyncio
nest_asyncio.apply()

In [69]:
if __name__ == '__main__':
    asyncio.run(main())


 Private key 4b63b8...cc8bc3 has been added to the queue. Queue №1 


 Private key 964cf0...daca7d has been added to the queue. Queue №2 


 Private key 11cf65...0edcfd has been added to the queue. Queue №3 


 Private key ed6e99...8e7027 has been added to the queue. Queue №4 

 ▶️ Before the start of the process №1 16 sec. 

 ▶️ Before the start of the process №2 15 sec. 

 ▶️ Before the start of the process №3 14 sec. 

 ▶️ Before the start of the process №4 11 sec. 


 Private key 3d6edd...9370db has been added to the queue. Queue №5 


 Private key 1d9d73...2bdd12 has been added to the queue. Queue №6 

Process №4 has been launched for ed6e99...8e7027 on the network Binance Smart Chain
Process №4 connection to Binance Smart Chain was established 

Process №3 has been launched for 11cf65...0edcfd on the network Binance Smart Chain
Process №3 connection to Binance Smart Chain was established 

Process №2 has been launched for 964cf0...daca7d on the network Binance Smart Chain
Proces

KeyboardInterrupt: 